In [ ]:
import numpy as np

In [ ]:
from PIL import Image
import os
import shutil
import Augmentor
from sklearn.model_selection import train_test_split

# Load images from the specified directory
imgs = []
base_path = "royce-hall"  # Original path where images are stored
valid_images = [".jpg"]
for f in os.listdir(base_path):  # Imports all jpg images into a list
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(Image.open(os.path.join(base_path, f)))

# Resize all images to 224 x 224 for RESNET
newsize = (224, 224)
resized_imgs = [i.resize(newsize) for i in imgs]

# Split images into train, validation, and test sets
train_val, test = train_test_split(resized_imgs, test_size=0.2)
train, val = train_test_split(train_val, test_size=0.25)  # 0.25 x 0.8 = 0.2

# Function to save images to a directory
def save_images(image_set, directory):
    if os.path.isdir(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)
    for k, img in enumerate(image_set):
        img.save(os.path.join(directory, f'image_{k}.jpg'))

# Define base directory for dataset
dataset_path = "dataset"

# Define paths for train and validation sets
train_path = os.path.join(dataset_path, 'train_set', 'royce-hall')
val_path = os.path.join(dataset_path, 'valid_set', 'royce-hall')

# Save train and validation images
save_images(train, train_path)
save_images(val, val_path)

# Augment images in the training set
output_path = os.path.join(train_path, 'output')
p = Augmentor.Pipeline(train_path, output_directory=output_path)
p.crop_random(probability=1, percentage_area=0.9)
p.resize(probability=1, width=224, height=224)
p.random_distortion(probability=0.3, grid_width=4, grid_height=4, magnitude=8)
p.skew(probability=0.4)
p.rotate_random_90(probability=0.8)
p.rotate(probability=0.6, max_left_rotation=25, max_right_rotation=25)
p.flip_left_right(probability=0.8)
p.sample(100)  # Generates 100 augmented images
